In [2]:
import csv
import numpy as np
import os
import pandas as pd
import time

In [ ]:
'''2017.4.21. indep disease와 gold disease간의 phenotype, ge similarity 구해야함.'''

In [8]:
gold_asso = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/new_predict/1_gold_learning_asso.tsv")
indep_asso = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/new_predict/3_indep_learning_asso.tsv")

In [26]:
gold_ge = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/new_predict/1_disease_disease_ge_sim_mat.tsv", index_col=0)
gold_phe = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/new_predict/1_disease_disease_phe_sim_mat.tsv", index_col=0)

In [21]:
print len(gold_ge)
print len(gold_phe)
print gold_ge.columns.tolist() == gold_phe.columns.tolist()
gold_ge.head(1)

233
233
True


,103780,608622,161900,145590,147530,157300,608088,140600,147480,600351,...,173200,115195,256700,601608,194070,115300,277350,240400,206200,250790
103780,1.0,0.93411,0.933826,0.982391,0.968759,0.984947,0.987788,0.960807,0.986936,0.979835,...,0.988072,0.981823,0.974439,0.98722,0.983811,0.986936,0.986936,0.988072,0.98722,0.98722


In [17]:
gold_di_id = gold_asso['disease_id'].drop_duplicates()
print len(gold_di_id)
indep_di_id = indep_asso['disease_id'].drop_duplicates()
print len(indep_di_id)

233
34


In [28]:
indep_ge_sim_mat = gold_ge.filter(items = indep_di_id.tolist(), axis = 0)
indep_phe_sim_mat = gold_phe.filter(items = indep_di_id.tolist(), axis = 0)

In [30]:
indep_ge_sim_mat.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/new_predict/3_indep_di_gold_di_geSim_mat.tsv", sep='\t')
indep_phe_sim_mat.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/new_predict/3_indep_di_gold_di_pheSim_mat.tsv", sep='\t')

In [8]:
indep_asso = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/10_PREDICT_association_indep.tsv")

In [4]:
indep_disease = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/8_gold_standard_disease_binary_matrix.tsv", index_col=0)

In [ ]:
#gold disease의 similarities. 여기서 indep만 빼기.
df_jac = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/9_gold_disease_similarity_genetic.tsv")
df_mim = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/9_gold_disease_similarity_mimminerc.tsv", index_col=0)

In [11]:
print len(indep_asso)
print len(indep_asso['drug'].drop_duplicates())
print len(indep_asso['disease'].drop_duplicates())
indep_asso.head()

2485
71
35


,drug,disease,association
0,DB00125,102300,0
1,DB00136,102300,0
2,DB00153,102300,0
3,DB00176,102300,0
4,DB00193,102300,0


In [22]:
indep_disease_id= indep_asso['disease'].drop_duplicates().tolist()

In [30]:
#genetic sim
temp = df_jac[df_jac['Disease_ID1'].isin(indep_disease_id)]
temp = temp[temp['Disease_ID2'].isin(indep_disease_id)]

In [32]:
print len(temp)
print len(temp['Disease_ID1'].drop_duplicates())
print len(temp['Disease_ID2'].drop_duplicates())

1225
35
35


In [34]:
temp.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/9_indep_disease_similarity_genetic.tsv", sep='\t', index=False)
temp.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/6_new_disease/9_indep_disease_similarity_genetic.tsv", sep='\t', index=False)

In [49]:
#phenotypic sim
str_disease_id = [str(x) for x in indep_disease_id]
temp2 = df_mim.filter(items = str_disease_id)
temp2 = temp2.filter(items = indep_disease_id, axis=0)

In [61]:
temp2.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/9_indep_disease_similarity_mimminerc.tsv", sep='\t')
temp2.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/6_new_disease/9_indep_disease_similarity_mimminerc.tsv", sep='\t')

In [73]:
#disease similairities 합치기
disease_ph = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/9_indep_disease_similarity_mimminerc.tsv", index_col=0)
disease_ge = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/9_indep_disease_similarity_genetic.tsv")

disease_ph2 = pd.DataFrame()
for i in disease_ph.columns.values:
    for j in disease_ph.index.values:
        temp_df = pd.DataFrame(data = [[int(i), int(j), disease_ph[[i]].loc[j].values[0]]], columns = ['Disease_ID1', 'Disease_ID2', 'Similarity_Score'] )
        disease_ph2 = disease_ph2.append(temp_df)
df_disease_sim = pd.merge(left = disease_ph2, right= disease_ge, left_on = ['Disease_ID1', 'Disease_ID2'], right_on = ['Disease_ID1', 'Disease_ID2'])
df_disease_sim = df_disease_sim.rename(columns = {'Similarity_Score_x':'PheSim', 'Similarity_Score_y':'GeSim'})
df_disease_sim.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/6_new_disease/9_indep_disease_similarity_whole.tsv", sep='\t', index=False)
df_disease_sim.to_csv("/DAS_Storage1/aschoi/data/Drug_Repositioning/6_new_disease/9_indep_disease_similarity_whole.tsv", sep='\t', index=False)